In [4]:
!pip install pandas numpy scikit-learn xgboost joblib
import pandas as pd


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
data_to_predict_df = pd.read_csv('data/raw/data_to_predict.csv')
data_to_predict_df.head()

,sessionId,eDate,channel,country,os,networkType,deviceType,publisherCategory,advertiserCategory,product,subProduct,campaign,advPackage,advMaturity,rate
0,9705699729732200,2017-12-17,smt,BR,ios,unknown,mobile,NaN,education,130000706,319881193,301604736,com.kawecki.onemind,4+,0.010
1,12802690671750800,2017-12-17,smt,KH,android,3g,mobile,music & audio,entertainment,130167110,com.fundevs.app.mediaconverter,300988528,NaN,low maturity,0.008
2,21347409150256300,2017-12-17,smt,ID,ios,not_wifi,mobile,NaN,travel,130285027,com.alcaydevs.idlfree,300228923,com.go-jek.ios,4+,0.020
3,23329125091887800,2017-12-17,inr,RU,android,3g,mobile,social,music & audio,625569,drug.vokrug,301060338,com.p74.player,parental guidance,0.022
4,28266843791174100,2017-12-17,smt,DE,android,wifi,mobile,photography,NaN,130356465,com.photo.editor.cartoon,301011313,NaN,everyone,0.025


In [6]:
data_to_predict_df.describe(include='all')

,sessionId,eDate,channel,country,os,networkType,deviceType,publisherCategory,advertiserCategory,product,subProduct,campaign,advPackage,advMaturity,rate
count,1.000000e+04,10000,10000,10000,10000,10000,10000,6646,9636,1.000000e+04,9928,10000,4196,10000,10000.000000
unique,NaN,1,5,46,2,4,2,46,33,NaN,975,247,36,12,NaN
top,NaN,2017-12-17,mpb,RU,android,3g,mobile,social,entertainment,NaN,drug.vokrug,300798390,com.p74.player,low maturity,NaN
freq,NaN,10000,3812,2538,8024,6967,9734,1618,3456,NaN,959,1182,1865,4494,NaN
mean,4.617623e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.376681e+08,NaN,NaN,NaN,NaN,0.026102
std,2.659635e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.079928e+08,NaN,NaN,NaN,NaN,0.024923
min,6.715025e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.955700e+04,NaN,NaN,NaN,NaN,0.003000
25%,2.326715e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.255690e+05,NaN,NaN,NaN,NaN,0.011000
50%,4.607486e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.300101e+08,NaN,NaN,NaN,NaN,0.020000
75%,6.925258e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.068310e+08,NaN,NaN,NaN,NaN,0.026000


In [26]:
import pickle
import joblib
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

prediction_df = data_to_predict_df.copy()

with open('models/v4/target_encoder.pkl', 'rb') as f:
    encoders = pickle.load(f)

model = joblib.load('models/v4.2/best_model_v4.joblib')

# Verify model loaded correctly
print("Model type:", type(model))
print("Is fitted?", hasattr(model, 'fitted_'))

# rename our prediction_df columns to match the training data for product, subproduct, campaign
prediction_df.rename(columns={'product': 'product_clean', 'subProduct': 'subProduct_clean', 'campaign': 'campaign_clean'}, inplace=True)
combined_metrics_cols = ['channel', 'country', 'os', 'networkType', 'deviceType', 
                        'advMaturity', 'publisherCategory', 'advertiserCategory', 
                        'product_clean', 'subProduct_clean', 'campaign_clean']

X_encoded_ctr = encoders['ctr'].transform(prediction_df[combined_metrics_cols])
X_encoded_cpm = encoders['cpm'].transform(prediction_df[combined_metrics_cols])

# # # Combine the encoded features
X = pd.concat([X_encoded_ctr, X_encoded_cpm], axis=1)
X.columns = [f'{col}_ctr' for col in X_encoded_ctr.columns] + [f'{col}_cpm' for col in X_encoded_cpm.columns]

predictions = model.predict(X)
print(predictions)
predictions_unscaled = encoders['target_scaler'].inverse_transform(predictions)

# Create predictions DataFrame
predictions_df = pd.DataFrame(predictions_unscaled, 
                            columns=['CTR_predicted', 'CPM_predicted'],
                            index=prediction_df.index)

# View results
print(predictions_df.head())

Model type: <class 'sklearn.multioutput.MultiOutputRegressor'>
Is fitted? False
[[ 0.34160852 -0.22288693]
 [ 0.92832863 -0.12561391]
 [ 0.0411304  -0.42109677]
 ...
 [-0.02505654 -0.6208558 ]
 [ 0.10882141  0.1731787 ]
 [ 0.03092713 -0.2982307 ]]
   CTR_predicted  CPM_predicted
0       0.045925       0.133040
1       0.098444       0.165340
2       0.019029       0.067223
3       0.049211       0.066519
4       0.008435       0.137806


In [30]:
# Add predictions to the original dataframe
data_to_predict_df['CTR_predicted'] = predictions_df['CTR_predicted']
data_to_predict_df['CPM_predicted'] = predictions_df['CPM_predicted']

# View the results with both original data and predictions
print(prediction_df[['rate', 'CTR_predicted', 'CPM_predicted']].head())

# Optional: Calculate some basic stats
print("\nPrediction Statistics:")
data_to_predict_df.head()

    rate  CTR_predicted  CPM_predicted
0  0.010       0.045925       0.133040
1  0.008       0.098444       0.165340
2  0.020       0.019029       0.067223
3  0.022       0.049211       0.066519
4  0.025       0.008435       0.137806

Prediction Statistics:


,sessionId,eDate,channel,country,os,networkType,deviceType,publisherCategory,advertiserCategory,product,subProduct,campaign,advPackage,advMaturity,rate,CTR_predicted,CPM_predicted
0,9705699729732200,2017-12-17,smt,BR,ios,unknown,mobile,NaN,education,130000706,319881193,301604736,com.kawecki.onemind,4+,0.010,0.045925,0.133040
1,12802690671750800,2017-12-17,smt,KH,android,3g,mobile,music & audio,entertainment,130167110,com.fundevs.app.mediaconverter,300988528,NaN,low maturity,0.008,0.098444,0.165340
2,21347409150256300,2017-12-17,smt,ID,ios,not_wifi,mobile,NaN,travel,130285027,com.alcaydevs.idlfree,300228923,com.go-jek.ios,4+,0.020,0.019029,0.067223
3,23329125091887800,2017-12-17,inr,RU,android,3g,mobile,social,music & audio,625569,drug.vokrug,301060338,com.p74.player,parental guidance,0.022,0.049211,0.066519
4,28266843791174100,2017-12-17,smt,DE,android,wifi,mobile,photography,NaN,130356465,com.photo.editor.cartoon,301011313,NaN,everyone,0.025,0.008435,0.137806


In [31]:
# Create the processed data directory if it doesn't exist
os.makedirs('data/processed', exist_ok=True)

# Save the dataframe with predictions to CSV
output_path = 'data/results/predictions_with_features.csv'
data_to_predict_df.to_csv(output_path, index=False)
print(f"Saved predictions to: {output_path}")


Saved predictions to: data/processed/predictions_with_features.csv
